<a href="https://colab.research.google.com/github/dayanakanagaraj/Sample/blob/master/Copy_of_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Pre-Trained Model

In [ ]:
!pip install -q transformers sentencepiece nltk rouge-score

from transformers import MarianMTModel, MarianTokenizer
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

english_sentences = [
    "hello", "how are you", "i am fine", "what is your name", "nice to meet you",
    "i love machine learning", "do you like pizza", "good morning", "thank you", "see you later"
]
french_sentences = [
    "bonjour", "comment ça va", "je vais bien", "quel est ton nom", "ravi de vous rencontrer",
    "j'aime l'apprentissage automatique", "aimes-tu la pizza", "bonjour", "merci", "à plus tard"
]

model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

results = []
for i, src in enumerate(english_sentences):
    inputs = tokenizer(src, return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    tgt = tokenizer.decode(translated[0], skip_special_tokens=True)

    reference = [nltk.word_tokenize(french_sentences[i])]
    prediction = nltk.word_tokenize(tgt)
    bleu = sentence_bleu(reference, prediction)
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True).score(
        ' '.join(reference[0]), ' '.join(prediction))

    results.append({
        "English": src,
        "French (Reference)": french_sentences[i],
        "Predicted French": tgt,
        "BLEU": round(bleu, 4),
        "ROUGE-1": round(rouge['rouge1'].fmeasure, 4),
        "ROUGE-2": round(rouge['rouge2'].fmeasure, 4)
    })

import pandas as pd
pd.DataFrame(results)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower o

,English,French (Reference),Predicted French,BLEU,ROUGE-1,ROUGE-2
0,hello,bonjour,Bonjour.,0.0,1.0000,0.0000
1,how are you,comment ça va,comment allez-vous,0.0,0.3333,0.0000
2,i am fine,je vais bien,Je vais bien.,0.0,1.0000,1.0000
3,what is your name,quel est ton nom,quel est votre nom,0.0,0.7500,0.3333
4,nice to meet you,ravi de vous rencontrer,Ravie de vous rencontrer.,0.0,1.0000,1.0000
5,i love machine learning,j'aime l'apprentissage automatique,J'adore l'apprentissage automatique,0.0,0.8000,0.5000
6,do you like pizza,aimes-tu la pizza,Tu aimes les pizzas ?,0.0,0.7500,0.0000
7,good morning,bonjour,Bonjour.,0.0,1.0000,0.0000
8,thank you,merci,Je vous remercie.,0.0,0.0000,0.0000
9,see you later,à plus tard,A tout à l'heure.,0.0,0.0000,0.0000


In [ ]:
# Step 1: Install and import libraries
!pip install -q datasets nltk rouge-score

from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
nltk.download('punkt')

# Step 2: Prepare a small dataset (English-French)
english_sentences = [
    "hello", "how are you", "i am fine", "what is your name", "nice to meet you",
    "i love machine learning", "do you like pizza", "good morning", "thank you", "see you later"
]

french_sentences = [
    "bonjour", "comment ça va", "je vais bien", "quel est ton nom", "ravi de vous rencontrer",
    "j'aime l'apprentissage automatique", "aimes-tu la pizza", "bonjour", "merci", "à plus tard"
]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
len(english_sentences)

10

In [ ]:


# Step 3: Tokenization
def tokenize(sentences, num_words=10000):
    tokenizer = Tokenizer(num_words=num_words, oov_token='<OOV>')
    tokenizer.fit_on_texts(sentences)
    tensor = tokenizer.texts_to_sequences(sentences)
    return tokenizer, pad_sequences(tensor, padding='post')

en_tokenizer, en_tensor = tokenize(english_sentences)
fr_tokenizer, fr_tensor = tokenize(french_sentences)
input_vocab_size = len(en_tokenizer.word_index) + 1
target_vocab_size = len(fr_tokenizer.word_index) + 1


In [ ]:
en_tensor

array([[ 4,  0,  0,  0],
       [ 5,  6,  2,  0],
       [ 3,  7,  8,  0],
       [ 9, 10, 11, 12],
       [13, 14, 15,  2],
       [ 3, 16, 17, 18],
       [19,  2, 20, 21],
       [22, 23,  0,  0],
       [24,  2,  0,  0],
       [25,  2, 26,  0]], dtype=int32)

In [ ]:
target_vocab_size

28

In [ ]:
fr_tokenizer.word_index

{'<OOV>': 1,
 'bonjour': 2,
 'comment': 3,
 'ça': 4,
 'va': 5,
 'je': 6,
 'vais': 7,
 'bien': 8,
 'quel': 9,
 'est': 10,
 'ton': 11,
 'nom': 12,
 'ravi': 13,
 'de': 14,
 'vous': 15,
 'rencontrer': 16,
 "j'aime": 17,
 "l'apprentissage": 18,
 'automatique': 19,
 'aimes': 20,
 'tu': 21,
 'la': 22,
 'pizza': 23,
 'merci': 24,
 'à': 25,
 'plus': 26,
 'tard': 27}

In [ ]:
# Step 4: Define transformer model
class Transformer(tf.keras.Model):
    def __init__(self, input_vocab, target_vocab, d_model, num_heads, dff, pe_input, pe_target):
        super().__init__()
        self.encoder_embedding = tf.keras.layers.Embedding(input_vocab, d_model)
        self.decoder_embedding = tf.keras.layers.Embedding(target_vocab, d_model)
        self.pos_encoding_input = self.positional_encoding(pe_input, d_model)
        self.pos_encoding_target = self.positional_encoding(pe_target, d_model)

        self.enc_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dec_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dense_proj = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.final_dense = tf.keras.layers.Dense(target_vocab)

    def positional_encoding(self, max_len, dm):
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(dm)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(dm))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        return tf.cast(angle_rads, dtype=tf.float32)

    def call(self, inputs, training=False):
        inp, tar = inputs
        enc = self.encoder_embedding(inp) + self.pos_encoding_input[:tf.shape(inp)[1]]
        dec = self.decoder_embedding(tar) + self.pos_encoding_target[:tf.shape(tar)[1]]
        enc_output = self.enc_layer(enc, enc)
        dec_output = self.dec_layer(dec, enc_output)
        final = self.dense_proj(dec_output)
        return self.final_dense(final)


In [ ]:


# Step 5: Compile and train
model = Transformer(input_vocab=input_vocab_size, target_vocab=target_vocab_size,
                    d_model=64, num_heads=4, dff=128, pe_input=en_tensor.shape[1], pe_target=fr_tensor.shape[1])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit([en_tensor, fr_tensor[:, :-1]], fr_tensor[:, 1:], epochs=20, batch_size=32)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.0000e+00 - loss: 3.3361
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4333 - loss: 3.2944
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.4333 - loss: 3.2225
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.4333 - loss: 3.1065
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.4333 - loss: 2.9315
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.4333 - loss: 2.7226
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4333 - loss: 2.6628
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.4333 - loss: 2.7312
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.4333 - loss: 2.5967
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4333 - loss: 2.5014
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.4333 - loss: 2.5073
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.4333 - loss: 2.52

In [ ]:

# Step 6: Translation function
def translate(sentence):
    seq = en_tokenizer.texts_to_sequences([sentence])
    padded = pad_sequences(seq, maxlen=en_tensor.shape[1], padding='post')
    decoder_input = np.zeros((1, fr_tensor.shape[1]), dtype=np.int32)
    decoder_input[0][0] = fr_tokenizer.word_index.get('<sos>', 1)

    for i in range(1, fr_tensor.shape[1]):
        output = model([padded, decoder_input[:, :-1]], training=False)
        pred_id = tf.argmax(output[0, i-1]).numpy()
        decoder_input[0][i] = pred_id
        if pred_id == fr_tokenizer.word_index.get('<eos>'):
            break

    return ' '.join([fr_tokenizer.index_word.get(idx, '') for idx in decoder_input[0] if idx != 0])


In [ ]:

# Step 7: Evaluation
results = []
for i in range(10):
    ref = [nltk.word_tokenize(french_sentences[i].replace('<sos>', '').replace('<eos>', '').strip())]
    pred = nltk.word_tokenize(translate(english_sentences[i]))
    bleu = sentence_bleu(ref, pred)
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True).score(' '.join(ref[0]), ' '.join(pred))
    results.append((english_sentences[i], french_sentences[i].replace('<sos>', '').replace('<eos>', '').strip(), ' '.join(pred), bleu, rouge['rouge1'].fmeasure, rouge['rouge2'].fmeasure))

import pandas as pd
pd.DataFrame(results, columns=["English", "French (Reference)", "Predicted French", "BLEU", "ROUGE-1", "ROUGE-2"])

,English,French (Reference),Predicted French,BLEU,ROUGE-1,ROUGE-2
0,hello,bonjour,< OOV >,0,0.0,0.0
1,how are you,comment ça va,< OOV >,0,0.0,0.0
2,i am fine,je vais bien,< OOV >,0,0.0,0.0
3,what is your name,quel est ton nom,< OOV >,0,0.0,0.0
4,nice to meet you,ravi de vous rencontrer,< OOV >,0,0.0,0.0
5,i love machine learning,j'aime l'apprentissage automatique,< OOV >,0,0.0,0.0
6,do you like pizza,aimes-tu la pizza,< OOV >,0,0.0,0.0
7,good morning,bonjour,< OOV >,0,0.0,0.0
8,thank you,merci,< OOV >,0,0.0,0.0
9,see you later,à plus tard,< OOV >,0,0.0,0.0
